In [12]:
# importing tool functions
import numpy as np
import matplotlib.pyplot as plt
from tools import utils, ml_tools
import pandas as pd
import numpy as np

# machine learning imports
from sklearn.model_selection import train_test_split
import tensorflow


In [13]:
#path = '/Users/jiayangzhang/Library/CloudStorage/OneDrive-ImperialCollegeLondon/year4/nlp-physicseducation/testfiles'
path        = '/Users/EfiaA/OneDrive - Imperial College London/Imperial academic work/University life/Y4\MSci project/Project_Coding/nlp-physicseducation/testfiles'
dir_txtfldr = '/Users/EfiaA/OneDrive - Imperial College London/Imperial academic work/University life/Y4/MSci project/Project_Coding/anonymised_reports/anonymised_reports/year_1_2017/cycle_1/txt'

In [14]:
# -- Get files ---
df_files = utils.build_files_dataframe(dir_txtfldr, 'GS_', '.txt')

# -- Get labels ---
df_labels = utils.build_labels_dataframe('data/labels_y1c1.xlsx')

# -- Merge dataframes --
df = pd.merge(df_files, df_labels, left_on='StudentID', right_on='StudentID')      # merged dataframe: StudentID, Content, ArgumentLevel, ReasoningLevel


In [15]:
# -- Bag of Words ---
wordvec_names, wordvec_counts= ml_tools.BoW(df['Content'].tolist())
y_b = df['ReasoningLevel'].tolist()

# -- Feature extraction: TF-IDF ---
X_t = ml_tools.tf_idf(df['Content'].tolist())
y_t = df['ReasoningLevel']


In [16]:
from sklearn.preprocessing import LabelBinarizer

# coding the labels so easily interpretable by the computer
lb = LabelBinarizer()
lb.fit(y_b)
y_b = lb.transform(y_b)



In [17]:
#splitting to data set
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(wordvec_counts, y_b , train_size = 0.8)

In [18]:
input = X_train_b.shape[1]
print(input)

3251


In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import numpy as np


In [20]:
maxlen = 100 

In [21]:
#embedding_dim = 50
model = Sequential()
model.add(layers.Dense(10, input_dim = input, activation  = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.add(layers.Flatten())
model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
model.build(input)
model.summary()
# we can see that we have 32520 parameters to train
# these are the weights of the embedding layer



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                32520     
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
 flatten (Flatten)           (None, 1)                 0         
                                                                 
Total params: 32,531
Trainable params: 32,531
Non-trainable params: 0
_________________________________________________________________


In [22]:
history = model.fit(X_train_b, y_train_b,epochs = 2500, verbose=True, validation_data=(X_test_b, y_test_b), batch_size=30 )
loss, accuracy = model.evaluate(X_train_b, y_train_b, verbose=False)
#print("Training Accuracy:{:.4f}".format(accuracy))

Epoch 1/2500
2/2 [==============================] - 1s 237ms/step - loss: 2.9544 - accuracy: 0.2647 - val_loss: 0.9736 - val_accuracy: 0.3056
Epoch 2/2500
2/2 [==============================] - 0s 33ms/step - loss: 0.9785 - accuracy: 0.3529 - val_loss: 0.6453 - val_accuracy: 0.6944
Epoch 3/2500
2/2 [==============================] - 0s 31ms/step - loss: 0.6284 - accuracy: 0.7353 - val_loss: 0.6160 - val_accuracy: 0.7500
Epoch 4/2500
2/2 [==============================] - 0s 33ms/step - loss: 0.6058 - accuracy: 0.7500 - val_loss: 0.6046 - val_accuracy: 0.7500
Epoch 5/2500
2/2 [==============================] - 0s 31ms/step - loss: 0.5945 - accuracy: 0.7500 - val_loss: 0.5971 - val_accuracy: 0.7500
Epoch 6/2500
2/2 [==============================] - 0s 31ms/step - loss: 0.5867 - accuracy: 0.7500 - val_loss: 0.5922 - val_accuracy: 0.7500
Epoch 7/2500
2/2 [==============================] - 0s 33ms/step - loss: 0.5820 - accuracy: 0.7500 - val_loss: 0.5894 - val_accuracy: 0.7500
Epoch 8/2500

In [30]:
score = model.evaluate(X_test_b, y_test_b, verbose =1)
print("Test Score", score[0])
print('Test Accuracy:', score[1])

1/1 [==============================] - 0s 29ms/step - loss: 0.5623 - accuracy: 0.7500
Test Score 0.5623348951339722
Test Accuracy: 0.75


In [23]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()

KeyError: 'acc'